In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.datasets import FashionMNIST as FashionMNISTDataset
import kornia
import matplotlib
import matplotlib.pyplot as plt
import captum
import math 
from cleverhans.torch.attacks.projected_gradient_descent import (
    projected_gradient_descent,
)

In [2]:
import gc
from captum.attr import *
import quantus

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [7]:
transformer = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
train_set = FashionMNISTDataset(root='./sample_data', train=True, transform=transformer, download=True)
test_set = FashionMNISTDataset(root='./sample_data', train=False, transform=transformer, download=True)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True, pin_memory=True) # num_workers=4,
test_loader = torch.utils.data.DataLoader(test_set, batch_size=32, pin_memory=True)

In [8]:
#for natural and adversarial LeNet Model 
class LeNet_normal(torch.nn.Module):
    """Network architecture from: https://github.com/ChawDoe/LeNet5-MNIST-PyTorch."""
    def __init__(self):
        super().__init__()
        self.conv_1 = torch.nn.Conv2d(1, 6, 5)
        self.pool_1 = torch.nn.MaxPool2d(2, 2)
        self.relu_1 = torch.nn.ReLU()
        self.conv_2 = torch.nn.Conv2d(6, 16, 5)
        self.pool_2 = torch.nn.MaxPool2d(2, 2)
        self.relu_2 = torch.nn.ReLU()
        self.fc_1 = torch.nn.Linear(256, 120)
        self.relu_3 = torch.nn.ReLU()
        self.fc_2 = torch.nn.Linear(120, 84)
        self.relu_4 = torch.nn.ReLU()
        self.fc_3 = torch.nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool_1(self.relu_1(self.conv_1(x)))
        x = self.pool_2(self.relu_2(self.conv_2(x)))
        x = x.view(x.shape[0], -1)
        x = self.relu_3(self.fc_1(x))
        x = self.relu_4(self.fc_2(x))
        x = self.fc_3(x)
        return x

# Train natural model 

In [9]:
model = LeNet_normal()
learning_rate = 0.001
epochs = 50 
criterion = torch.nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
def evaluate_model(model, data, device):
    """Evaluate torch model."""
    model.eval()
    logits = torch.Tensor().to(device)
    targets = torch.LongTensor().to(device)

    with torch.no_grad():
        for images, labels in data:
            images, labels = images.to(device), labels.to(device)
            logits = torch.cat([logits, model(images)])
            targets = torch.cat([targets, labels])
    
    return torch.nn.functional.softmax(logits, dim=1), targets

In [9]:
# Train adversarial model 

def train_natural_model(model, epochs):
    model.train()
    for epoch in range(epochs):
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            logits = model(images)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

        # Evaluate model!
        if epochs%10==0:
            predictions, labels = evaluate_model(model, test_loader, device)
            test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())
            print(f"Epoch {epoch+1}/{epochs} - test accuracy: {(100 * test_acc):.2f}% and CE loss {loss.item():.2f}")
    return model

In [15]:
model_nat = train_natural_model(model=model.to(device),
                    epochs=epochs)

Epoch 1/50 - test accuracy: 82.18% and CE loss 0.54
Epoch 2/50 - test accuracy: 86.96% and CE loss 0.21
Epoch 3/50 - test accuracy: 87.11% and CE loss 0.45
Epoch 4/50 - test accuracy: 88.09% and CE loss 0.22
Epoch 5/50 - test accuracy: 88.37% and CE loss 0.06
Epoch 6/50 - test accuracy: 88.54% and CE loss 0.36
Epoch 7/50 - test accuracy: 89.57% and CE loss 0.29
Epoch 8/50 - test accuracy: 89.53% and CE loss 0.27
Epoch 9/50 - test accuracy: 89.68% and CE loss 0.07
Epoch 10/50 - test accuracy: 90.11% and CE loss 0.07
Epoch 11/50 - test accuracy: 90.13% and CE loss 0.12
Epoch 12/50 - test accuracy: 89.49% and CE loss 0.21
Epoch 13/50 - test accuracy: 89.66% and CE loss 0.21
Epoch 14/50 - test accuracy: 90.20% and CE loss 0.11
Epoch 15/50 - test accuracy: 90.42% and CE loss 0.14
Epoch 16/50 - test accuracy: 90.26% and CE loss 0.08
Epoch 17/50 - test accuracy: 89.05% and CE loss 0.19
Epoch 18/50 - test accuracy: 89.70% and CE loss 0.12
Epoch 19/50 - test accuracy: 90.22% and CE loss 0.17
Ep

In [17]:
# Model to GPU and eval mode.
model_nat.to(device)
model_nat.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_nat, test_loader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")

Model test accuracy: 89.68%


In [18]:
#save adv model 
path = 'FMNIST_Natural_model.pth'
torch.save(model_nat.state_dict(), path)

# Train adversarial models

In [10]:
model = LeNet_normal()

In [11]:
learning_rate = 1e-4 
epochs = 40
eps = [0.01, 0.03, 0.06, 0.1, 0.3, 0.5]

In [12]:
criterion = torch.nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [53]:
def evaluate_model(model, data, device):
    """Evaluate torch model."""
    model.eval()
    logits = torch.Tensor().to(device)
    targets = torch.LongTensor().to(device)

    with torch.no_grad():
        for images, labels in data:
            images, labels = images.to(device), labels.to(device)
            logits = torch.cat([logits, model(images)])
            targets = torch.cat([targets, labels])
    
    return torch.nn.functional.softmax(logits, dim=1), targets

In [54]:
# Train adversarial model 

def train_Adversarial_model(model, epsilon, epochs):
    model.train()
    eps = epsilon
    for epoch in range(epochs):
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            images = projected_gradient_descent(model, images, eps, eps/10, 40, np.inf)
            optimizer.zero_grad()
            logits = model(images)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

        # Evaluate model!
        if epochs%5==0:
            predictions, labels = evaluate_model(model, test_loader, device)
            test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())
            print(f"Epoch {epoch+1}/{epochs} - test accuracy: {(100 * test_acc):.2f}% and CE loss {loss.item():.2f}")
    return model

In [55]:
model_adv = train_Adversarial_model(model=model.to(device),
                    epsilon = eps[0], 
                    epochs=epochs)

Epoch 1/40 - test accuracy: 72.87% and CE loss 0.40
Epoch 2/40 - test accuracy: 74.95% and CE loss 0.71
Epoch 3/40 - test accuracy: 78.18% and CE loss 0.64
Epoch 4/40 - test accuracy: 79.76% and CE loss 0.53
Epoch 5/40 - test accuracy: 80.05% and CE loss 0.82
Epoch 6/40 - test accuracy: 81.13% and CE loss 0.75
Epoch 7/40 - test accuracy: 82.28% and CE loss 0.68
Epoch 8/40 - test accuracy: 82.49% and CE loss 0.38
Epoch 9/40 - test accuracy: 83.52% and CE loss 0.39
Epoch 10/40 - test accuracy: 84.21% and CE loss 0.40
Epoch 11/40 - test accuracy: 84.43% and CE loss 0.51
Epoch 12/40 - test accuracy: 83.78% and CE loss 0.71
Epoch 13/40 - test accuracy: 85.48% and CE loss 0.39
Epoch 14/40 - test accuracy: 85.33% and CE loss 0.48
Epoch 15/40 - test accuracy: 85.64% and CE loss 0.37
Epoch 16/40 - test accuracy: 86.01% and CE loss 0.41
Epoch 17/40 - test accuracy: 85.78% and CE loss 0.75
Epoch 18/40 - test accuracy: 86.25% and CE loss 0.26
Epoch 19/40 - test accuracy: 86.60% and CE loss 0.53
Ep

In [56]:
# Model to GPU and eval mode.
model_adv.to(device)
model_adv.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_adv, test_loader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")

Model test accuracy: 88.27%


In [57]:
#save adv model 
path = 'Eps_A_Adv_model.pth'
torch.save(model_adv.state_dict(), path)

# Train model for second epsilon setting

In [59]:
model = LeNet_normal()

In [60]:
criterion = torch.nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [61]:
model_adv = train_Adversarial_model(model=model.to(device),
                    epsilon = eps[1], 
                    epochs=epochs)

Epoch 1/40 - test accuracy: 71.55% and CE loss 0.89
Epoch 2/40 - test accuracy: 74.04% and CE loss 0.64
Epoch 3/40 - test accuracy: 77.01% and CE loss 0.68
Epoch 4/40 - test accuracy: 78.44% and CE loss 0.51
Epoch 5/40 - test accuracy: 79.01% and CE loss 0.73
Epoch 6/40 - test accuracy: 80.04% and CE loss 0.55
Epoch 7/40 - test accuracy: 80.94% and CE loss 0.51
Epoch 8/40 - test accuracy: 82.09% and CE loss 0.62
Epoch 9/40 - test accuracy: 82.64% and CE loss 0.49
Epoch 10/40 - test accuracy: 82.78% and CE loss 0.63
Epoch 11/40 - test accuracy: 82.64% and CE loss 0.40
Epoch 12/40 - test accuracy: 83.71% and CE loss 0.78
Epoch 13/40 - test accuracy: 83.44% and CE loss 0.62
Epoch 14/40 - test accuracy: 83.80% and CE loss 0.61
Epoch 15/40 - test accuracy: 84.39% and CE loss 0.52
Epoch 16/40 - test accuracy: 84.33% and CE loss 0.71
Epoch 17/40 - test accuracy: 84.10% and CE loss 0.39
Epoch 18/40 - test accuracy: 85.05% and CE loss 0.40
Epoch 19/40 - test accuracy: 84.77% and CE loss 0.50
Ep

In [62]:
# Model to GPU and eval mode.
model_adv.to(device)
model_adv.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_adv, test_loader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")

Model test accuracy: 86.66%


In [63]:
#save adv model 
path = 'Eps_B_Adv_model.pth'
torch.save(model_adv.state_dict(), path)

# Train feature map smoothed models

In [85]:
learning_rate = 1e-4 
epochs = 40
eps = [0.01, 0.03, 0.06, 0.1, 0.3]

# Mean filtering

In [86]:
class filter_denoising_block(torch.nn.Module):
    ''' Simple filters as denoising block'''
    def __init__(self, in_planes, ksize, filter_type): 
        super().__init__()
        self.in_planes = in_planes
        self.ksize = ksize
        self.filter_type = filter_type 
        self.conv = nn.Conv2d(in_channels=in_planes, out_channels=in_planes, kernel_size=1, stride=1, padding=0)
    
    def forward(self, x):
        if self.filter_type == 'Median':
            x_denoised = kornia.filters.median_blur(x, (self.ksize, self.ksize))
        elif self.filter_type == 'Mean':
            x_denoised = kornia.filters.box_blur(x, (self.ksize, self.ksize))
        elif self.filter_type == 'Gaussian':
            x_denoised = kornia.filters.gaussian_blur2d(x, (self.ksize, self.ksize), (0.3 * ((x.shape[3] - 1) * 0.5 - 1) + 0.8, 0.3 * ((x.shape[2] - 1) * 0.5 - 1) + 0.8))
        new_x = x + self.conv(x_denoised)
        return new_x

In [87]:
class non_local_denoising_block(torch.nn.Module): 
    def __init__(self, in_channels, inter_channels = None, mode = 'embedded', bn_layer = True):
        
        """ in_channels: original channel size (1024 in the paper) #depends on the test dataset
            inter_channels: channel size inside the block if not specifed reduced to half (512 in the paper)
            mode: supports Gaussian, Embedded Gaussian, Dot Product, and Concatenation
            dimension: can be 1 (temporal), 2 (spatial), 3 (spatiotemporal)
            bn_layer: whether to add batch norm
            """
        
        super().__init__()
        self.in_channels = in_channels
        self.inter_channels = inter_channels
        self.mode = mode 
        self.inter_channels = inter_channels
        
         # the channel size is reduced to half inside the block
        if self.inter_channels is None:
            self.inter_channels = in_channels // 2
            if self.inter_channels == 0:
                self.inter_channels = 1
        
        
        #dimension=2 for images
        conv_nd = nn.Conv2d 
        max_pool_layer = nn.MaxPool2d(kernel_size=(2,2))
        bn = nn.BatchNorm2d
        
        self.g = conv_nd(in_channels=self.in_channels, out_channels=self.inter_channels, kernel_size=1)
        
        # add BatchNorm layer after the last conv layer
        if bn_layer:
            self.W_z = nn.Sequential(
                    conv_nd(in_channels=self.inter_channels, out_channels=self.in_channels, kernel_size=1),
                    bn(self.in_channels)
                )
            # from section 4.1 of the paper, initializing params of BN ensures that the initial state of non-local block is identity mapping
            nn.init.constant_(self.W_z[1].weight, 0)
            nn.init.constant_(self.W_z[1].bias, 0)
        else:
            self.W_z = conv_nd(in_channels=self.inter_channels, out_channels=self.in_channels, kernel_size=1)
            
            # from section 3.3 of the paper by initializing Wz to 0, this block can be inserted to any existing architecture
            nn.init.constant_(self.W_z.weight, 0)
            nn.init.constant_(self.W_z.bias, 0)
            
            
        # define theta and phi for all operations except gaussian
        if self.mode == "embedded" or self.mode == "dot" or self.mode == "concatenate":
            self.theta = conv_nd(in_channels=self.in_channels, out_channels=self.inter_channels, kernel_size=1)
            self.phi = conv_nd(in_channels=self.in_channels, out_channels=self.inter_channels, kernel_size=1)
        
        if self.mode == "concatenate":
            self.W_f = nn.Sequential(
                    nn.Conv2d(in_channels=self.inter_channels * 2, out_channels=1, kernel_size=1),
                    nn.ReLU()
                )
    
    def forward(self, x):
        batch_size = x.size(0)
        
        g_x = self.g(x).view(batch_size, self.inter_channels, -1)
        g_x = g_x.permute(0, 2, 1)
        
        if self.mode == "nonlocal_gaussian":
            theta_x = x.view(batch_size, self.in_channels, -1)
            phi_x = x.view(batch_size, self.in_channels, -1)
            theta_x = theta_x.permute(0, 2, 1)
            f = torch.matmul(theta_x, phi_x)

        elif self.mode == "embedded" or self.mode == "dot":
            theta_x = self.theta(x).view(batch_size, self.inter_channels, -1)
            phi_x = self.phi(x).view(batch_size, self.inter_channels, -1)
            theta_x = theta_x.permute(0, 2, 1)
            f = torch.matmul(theta_x, phi_x)

        elif self.mode == "concatenate":
            theta_x = self.theta(x).view(batch_size, self.inter_channels, -1, 1)
            phi_x = self.phi(x).view(batch_size, self.inter_channels, 1, -1)
            
            h = theta_x.size(2)
            w = phi_x.size(3)
            theta_x = theta_x.repeat(1, 1, 1, w)
            phi_x = phi_x.repeat(1, 1, h, 1)
            
            concat = torch.cat([theta_x, phi_x], dim=1)
            f = self.W_f(concat)
            f = f.view(f.size(0), f.size(2), f.size(3))
        
        if self.mode == "nonlocal_gaussian" or self.mode == "embedded":
            f_div_C = F.softmax(f, dim=-1)
        elif self.mode == "dot" or self.mode == "concatenate":
            N = f.size(-1) # number of position in x
            f_div_C = f / N
        
        y = torch.matmul(f_div_C, g_x)
        
        # contiguous here just allocates contiguous chunk of memory
        y = y.permute(0, 2, 1).contiguous()
        y = y.view(batch_size, self.inter_channels, *x.size()[2:])
        
        W_y = self.W_z(y)
        # residual connection
        z = W_y + x

        return z

In [88]:
class LeNetDenoise(torch.nn.Module):
    """Network architecture from: https://github.com/ChawDoe/LeNet5-MNIST-PyTorch."""
    def __init__(self, denoise="Mean", ksize=1):
        super().__init__()
        self.conv_1 = torch.nn.Conv2d(1, 6, 5)
        self.pool_1 = torch.nn.MaxPool2d(2, 2)
        self.relu_1 = torch.nn.ReLU()
        
        # add a denoising layer
        if denoise == "Mean" or denoise == "Median" or denoise == "Gaussian":
            self.denoise_block1 = filter_denoising_block(in_planes = 6, ksize = ksize, filter_type=denoise)
            self.denoise_block2 = filter_denoising_block(in_planes = 16, ksize = ksize, filter_type=denoise)
        
        elif denoise == "concatenate" or denoise == "dot" or denoise == "embedded" or denoise == "nonlocal_gaussian" :
            self.denoise_block1 = non_local_denoising_block(in_channels = 6,  mode=denoise) 
            self.denoise_block2 = non_local_denoising_block(in_channels = 16, mode=denoise) 
            
        self.conv_2 = torch.nn.Conv2d(6, 16, 5)
        self.pool_2 = torch.nn.MaxPool2d(2, 2)
        self.relu_2 = torch.nn.ReLU()
    
    
        self.fc_1 = torch.nn.Linear(256, 120)
        self.relu_3 = torch.nn.ReLU()
        self.fc_2 = torch.nn.Linear(120, 84)
        self.relu_4 = torch.nn.ReLU()
        self.fc_3 = torch.nn.Linear(84, 10)
    
    def forward(self, x):
        x = self.pool_1(self.relu_1(self.conv_1(x)))
        x = self.denoise_block1(x) 
        x = self.pool_2(self.relu_2(self.conv_2(x)))
        #x = self.denoise_block2(x)
        
        x = x.view(x.shape[0], -1)
        x = self.relu_3(self.fc_1(x))
        x = self.relu_4(self.fc_2(x))
        x = self.fc_3(x)
        return x

In [89]:
denoise = "Mean"
model = LeNetDenoise(denoise=denoise, ksize=3)

In [90]:
criterion = torch.nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [91]:
model_adv = train_Adversarial_model(model=model.to(device),
                    epsilon = eps[0], 
                    epochs=epochs)

Epoch 1/40 - test accuracy: 73.57% and CE loss 0.61
Epoch 2/40 - test accuracy: 73.51% and CE loss 0.70
Epoch 3/40 - test accuracy: 77.72% and CE loss 0.67
Epoch 4/40 - test accuracy: 79.01% and CE loss 0.60
Epoch 5/40 - test accuracy: 80.37% and CE loss 0.43
Epoch 6/40 - test accuracy: 80.03% and CE loss 0.73
Epoch 7/40 - test accuracy: 81.94% and CE loss 0.59
Epoch 8/40 - test accuracy: 82.35% and CE loss 0.43
Epoch 9/40 - test accuracy: 83.03% and CE loss 0.33
Epoch 10/40 - test accuracy: 83.28% and CE loss 0.24
Epoch 11/40 - test accuracy: 84.05% and CE loss 0.36
Epoch 12/40 - test accuracy: 84.50% and CE loss 0.34
Epoch 13/40 - test accuracy: 84.81% and CE loss 0.45
Epoch 14/40 - test accuracy: 85.49% and CE loss 0.54
Epoch 15/40 - test accuracy: 85.15% and CE loss 0.38
Epoch 16/40 - test accuracy: 85.66% and CE loss 0.33
Epoch 17/40 - test accuracy: 86.06% and CE loss 0.39
Epoch 18/40 - test accuracy: 86.31% and CE loss 0.42
Epoch 19/40 - test accuracy: 86.34% and CE loss 0.31
Ep

In [92]:
# Model to GPU and eval mode.
model_adv.to(device)
model_adv.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_adv, test_loader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")

Model test accuracy: 88.27%


In [93]:
#save adv model 
path = 'Eps_A_Mean_Adv_model.pth'
torch.save(model_adv.state_dict(), path)

# Median

In [123]:
learning_rate = 1e-4 
epochs = 40
eps = [0.01, 0.03, 0.06, 0.1, 0.3]

In [124]:
denoise = "Median"
model = LeNetDenoise(denoise=denoise, ksize=3)


criterion = torch.nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

model_adv = train_Adversarial_model(model=model.to(device),
                    epsilon = eps[0], 
                    epochs=epochs)

Epoch 1/40 - test accuracy: 72.62% and CE loss 0.76
Epoch 2/40 - test accuracy: 74.69% and CE loss 0.52
Epoch 3/40 - test accuracy: 77.50% and CE loss 0.49
Epoch 4/40 - test accuracy: 78.90% and CE loss 0.44
Epoch 5/40 - test accuracy: 80.35% and CE loss 0.32
Epoch 6/40 - test accuracy: 81.53% and CE loss 0.47
Epoch 7/40 - test accuracy: 80.89% and CE loss 0.60
Epoch 8/40 - test accuracy: 82.29% and CE loss 0.53
Epoch 9/40 - test accuracy: 83.53% and CE loss 0.48
Epoch 10/40 - test accuracy: 83.53% and CE loss 0.67
Epoch 11/40 - test accuracy: 84.28% and CE loss 0.56
Epoch 12/40 - test accuracy: 83.83% and CE loss 0.52
Epoch 13/40 - test accuracy: 84.14% and CE loss 0.33
Epoch 14/40 - test accuracy: 84.55% and CE loss 0.65
Epoch 15/40 - test accuracy: 85.28% and CE loss 0.60
Epoch 16/40 - test accuracy: 85.44% and CE loss 0.38
Epoch 17/40 - test accuracy: 85.53% and CE loss 0.33
Epoch 18/40 - test accuracy: 85.84% and CE loss 0.74
Epoch 19/40 - test accuracy: 86.05% and CE loss 0.41
Ep

In [125]:
# Model to GPU and eval mode.
model_adv.to(device)
model_adv.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_adv, test_loader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")


#save adv model 
path = 'Eps_A_Median_Adv_model.pth'
torch.save(model_adv.state_dict(), path)

Model test accuracy: 88.06%


In [127]:
del model 
del model_adv

# Gaussian

In [144]:
learning_rate = 1e-4 
epochs = 40
eps = [0.01, 0.03, 0.06, 0.1, 0.3]

In [145]:
denoise = "Gaussian"
model = LeNetDenoise(denoise=denoise, ksize=3)


criterion = torch.nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

model_adv = train_Adversarial_model(model=model.to(device),
                    epsilon = eps[0], 
                    epochs=epochs)

Epoch 1/40 - test accuracy: 73.27% and CE loss 1.16
Epoch 2/40 - test accuracy: 76.40% and CE loss 0.64
Epoch 3/40 - test accuracy: 78.31% and CE loss 0.52
Epoch 4/40 - test accuracy: 79.00% and CE loss 0.77
Epoch 5/40 - test accuracy: 80.05% and CE loss 0.41
Epoch 6/40 - test accuracy: 81.77% and CE loss 0.84
Epoch 7/40 - test accuracy: 82.59% and CE loss 0.48
Epoch 8/40 - test accuracy: 83.27% and CE loss 0.47
Epoch 9/40 - test accuracy: 84.12% and CE loss 0.40
Epoch 10/40 - test accuracy: 84.40% and CE loss 0.35
Epoch 11/40 - test accuracy: 84.34% and CE loss 0.41
Epoch 12/40 - test accuracy: 84.85% and CE loss 0.46
Epoch 13/40 - test accuracy: 85.26% and CE loss 0.29
Epoch 14/40 - test accuracy: 85.97% and CE loss 0.52
Epoch 15/40 - test accuracy: 86.00% and CE loss 0.23
Epoch 16/40 - test accuracy: 86.18% and CE loss 0.45
Epoch 17/40 - test accuracy: 86.41% and CE loss 0.39
Epoch 18/40 - test accuracy: 86.54% and CE loss 0.28
Epoch 19/40 - test accuracy: 86.51% and CE loss 0.33
Ep

In [146]:
# Model to GPU and eval mode.
model_adv.to(device)
model_adv.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_adv, test_loader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")


#save adv model 
path = 'Eps_A_Gaussian_Adv_model.pth'
torch.save(model_adv.state_dict(), path)

Model test accuracy: 88.71%


In [148]:
del model 
del model_adv

# Embedded 

In [165]:
learning_rate = 1e-4 
epochs = 40
eps = [0.01, 0.03, 0.06, 0.1, 0.3]

In [167]:
model = LeNetDenoise(denoise="embedded")
criterion = torch.nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

model_adv = train_Adversarial_model(model=model.to(device),
                    epsilon = eps[0], 
                    epochs=epochs)

Epoch 1/40 - test accuracy: 70.21% and CE loss 0.83
Epoch 2/40 - test accuracy: 75.24% and CE loss 0.46
Epoch 3/40 - test accuracy: 76.58% and CE loss 0.45
Epoch 4/40 - test accuracy: 77.90% and CE loss 0.29
Epoch 5/40 - test accuracy: 79.26% and CE loss 0.59
Epoch 6/40 - test accuracy: 80.44% and CE loss 0.69
Epoch 7/40 - test accuracy: 81.03% and CE loss 0.45
Epoch 8/40 - test accuracy: 82.07% and CE loss 0.23
Epoch 9/40 - test accuracy: 82.57% and CE loss 0.63
Epoch 10/40 - test accuracy: 83.29% and CE loss 0.48
Epoch 11/40 - test accuracy: 83.75% and CE loss 0.31
Epoch 12/40 - test accuracy: 84.82% and CE loss 0.50
Epoch 13/40 - test accuracy: 83.92% and CE loss 0.38
Epoch 14/40 - test accuracy: 85.60% and CE loss 0.40
Epoch 15/40 - test accuracy: 85.95% and CE loss 0.54
Epoch 16/40 - test accuracy: 85.71% and CE loss 0.39
Epoch 17/40 - test accuracy: 86.28% and CE loss 0.57
Epoch 18/40 - test accuracy: 86.11% and CE loss 0.40
Epoch 19/40 - test accuracy: 86.32% and CE loss 0.34
Ep

In [168]:
# Model to GPU and eval mode.
model_adv.to(device)
model_adv.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_adv, test_loader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")


#save adv model 
path = 'Eps_A_Embed_Adv_model.pth'
torch.save(model_adv.state_dict(), path)

Model test accuracy: 88.56%


In [170]:
del model 
del model_adv

# Non Local Gaussian

In [188]:
model = LeNetDenoise(denoise="nonlocal_gaussian")
criterion = torch.nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

model_adv = train_Adversarial_model(model=model.to(device),
                    epsilon = eps[0], 
                    epochs=epochs)

Epoch 1/40 - test accuracy: 72.38% and CE loss 0.58
Epoch 2/40 - test accuracy: 74.40% and CE loss 0.85
Epoch 3/40 - test accuracy: 76.31% and CE loss 0.54
Epoch 4/40 - test accuracy: 78.07% and CE loss 0.68
Epoch 5/40 - test accuracy: 78.85% and CE loss 0.76
Epoch 6/40 - test accuracy: 79.78% and CE loss 0.65
Epoch 7/40 - test accuracy: 81.28% and CE loss 0.52
Epoch 8/40 - test accuracy: 81.98% and CE loss 0.44
Epoch 9/40 - test accuracy: 82.68% and CE loss 0.46
Epoch 10/40 - test accuracy: 83.05% and CE loss 0.48
Epoch 11/40 - test accuracy: 83.95% and CE loss 0.55
Epoch 12/40 - test accuracy: 84.24% and CE loss 0.33
Epoch 13/40 - test accuracy: 84.85% and CE loss 0.35
Epoch 14/40 - test accuracy: 84.77% and CE loss 0.63
Epoch 15/40 - test accuracy: 84.72% and CE loss 0.52
Epoch 16/40 - test accuracy: 85.07% and CE loss 0.58
Epoch 17/40 - test accuracy: 85.39% and CE loss 0.28
Epoch 18/40 - test accuracy: 86.05% and CE loss 0.29
Epoch 19/40 - test accuracy: 85.97% and CE loss 0.35
Ep

In [189]:
# Model to GPU and eval mode.
model_adv.to(device)
model_adv.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_adv, test_loader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")


#save adv model 
path = 'Eps_A_NonLocal_Adv_model.pth'
torch.save(model_adv.state_dict(), path)

Model test accuracy: 87.29%


In [191]:
del model
del model_adv